In [1]:
import pandas as pd
import numpy as np
from sodapy import Socrata
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [4]:
# Initialize the Socrata client and fetch data
client = Socrata("data.ny.gov", "UySBT3QA7VB8OpdyYVybWrBiN")
results = client.get("wujg-7c2s", limit=5000000)
results


[{'transit_timestamp': '2023-09-09T21:00:00.000',
  'transit_mode': 'subway',
  'station_complex_id': '372',
  'station_complex': 'Longwood Av (6)',
  'borough': 'Bronx',
  'payment_method': 'metrocard',
  'fare_class_category': 'Metrocard - Unlimited 30-Day',
  'ridership': '1.0',
  'transfers': '0.0',
  'latitude': '40.816105',
  'longitude': '-73.89644',
  'georeference': {'type': 'Point', 'coordinates': [-73.89644, 40.816105]},
  ':@computed_region_kjdx_g34t': '2032',
  ':@computed_region_yamh_8v7k': '307',
  ':@computed_region_wbg7_3whc': '651'},
 {'transit_timestamp': '2023-09-09T21:00:00.000',
  'transit_mode': 'subway',
  'station_complex_id': '384',
  'station_complex': 'Burnside Av (4)',
  'borough': 'Bronx',
  'payment_method': 'metrocard',
  'fare_class_category': 'Metrocard - Fair Fare',
  'ridership': '12.0',
  'transfers': '1.0',
  'latitude': '40.853455',
  'longitude': '-73.907684',
  'georeference': {'type': 'Point', 'coordinates': [-73.907684, 40.853455]},
  ':@compu

In [5]:

# Convert results to DataFrame
df = pd.DataFrame.from_records(results)

# Convert 'transit_timestamp' to datetime
df['transit_timestamp'] = pd.to_datetime(df['transit_timestamp'])

# Convert 'ridership' to numeric
df['ridership'] = pd.to_numeric(df['ridership'], errors='coerce')

print(df.head())
df

    transit_timestamp transit_mode station_complex_id      station_complex  \
0 2023-09-09 21:00:00       subway                372      Longwood Av (6)   
1 2023-09-09 21:00:00       subway                384      Burnside Av (4)   
2 2023-09-09 21:00:00       subway                386       Mt Eden Av (4)   
3 2023-09-09 21:00:00       subway                 39  Bay Ridge-95 St (R)   
4 2023-09-09 21:00:00       subway                392       125 St (4,5,6)   

     borough payment_method           fare_class_category  ridership  \
0      Bronx      metrocard  Metrocard - Unlimited 30-Day        1.0   
1      Bronx      metrocard         Metrocard - Fair Fare       12.0   
2      Bronx      metrocard         Metrocard - Full Fare        5.0   
3   Brooklyn      metrocard  Metrocard - Unlimited 30-Day        5.0   
4  Manhattan      metrocard   Metrocard - Unlimited 7-Day       31.0   

  transfers   latitude   longitude  \
0       0.0  40.816105   -73.89644   
1       1.0  40.853455

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
0,2023-09-09 21:00:00,subway,372,Longwood Av (6),Bronx,metrocard,Metrocard - Unlimited 30-Day,1.0,0.0,40.816105,-73.89644,"{'type': 'Point', 'coordinates': [-73.89644, 4...",2032,307,651
1,2023-09-09 21:00:00,subway,384,Burnside Av (4),Bronx,metrocard,Metrocard - Fair Fare,12.0,1.0,40.853455,-73.907684,"{'type': 'Point', 'coordinates': [-73.907684, ...",2032,307,649
2,2023-09-09 21:00:00,subway,386,Mt Eden Av (4),Bronx,metrocard,Metrocard - Full Fare,5.0,0.0,40.844433,-73.91469,"{'type': 'Point', 'coordinates': [-73.91469, 4...",2032,307,648
3,2023-09-09 21:00:00,subway,39,Bay Ridge-95 St (R),Brooklyn,metrocard,Metrocard - Unlimited 30-Day,5.0,0.0,40.616623,-74.030876,"{'type': 'Point', 'coordinates': [-74.030876, ...",2090,894,887
4,2023-09-09 21:00:00,subway,392,"125 St (4,5,6)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,31.0,0.0,40.80414,-73.93759,"{'type': 'Point', 'coordinates': [-73.93759, 4...",2095,749,794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4999995,2022-03-08 01:00:00,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Full Fare,2.0,0.0,40.60321807861328,-73.97235870361328,"{'type': 'Point', 'coordinates': [-73.97235870...",2090,894,902
4999996,2022-03-03 20:00:00,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Fair Fare,2.0,0.0,40.60321807861328,-73.97235870361328,"{'type': 'Point', 'coordinates': [-73.97235870...",2090,894,902
4999997,2022-11-30 06:00:00,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Fair Fare,8.0,0.0,40.60321807861328,-73.97235870361328,"{'type': 'Point', 'coordinates': [-73.97235870...",2090,894,902
4999998,2022-09-10 14:00:00,subway,250,Kings Hwy (F),Brooklyn,metrocard,Metrocard - Other,8.0,0.0,40.60321807861328,-73.97235870361328,"{'type': 'Point', 'coordinates': [-73.97235870...",2090,894,902


In [6]:
def get_average_ridership_by_station_and_time(dataframe, station_id, query_time):
    """
    Retrieve the average ridership for a given station and time, across the same day of the week.
    
    :param dataframe: DataFrame containing the MTA data
    :param station_id: ID or name of the station complex
    :param query_time: datetime or string representing the time to query (e.g., '08:00:00')
    :return: Average ridership for the given time and day of the week
    """
    # Ensure query_time is a datetime object for consistency
    if isinstance(query_time, str):
        query_time = pd.to_datetime(query_time)

    # Filter data for the given station
    station_filtered = dataframe[
        (dataframe['station_complex_id'] == station_id) |
        (dataframe['station_complex'].str.contains(station_id, case=False, na=False))
    ].copy()

    # Safely create new columns using loc
    station_filtered.loc[:, 'day_of_week'] = station_filtered['transit_timestamp'].dt.dayofweek
    station_filtered.loc[:, 'hour'] = station_filtered['transit_timestamp'].dt.hour

    # Filter by day of the week and hour
    day_hour_filtered = station_filtered[
        (station_filtered['day_of_week'] == query_time.dayofweek) &
        (station_filtered['hour'] == query_time.hour)
    ]

    # Calculate the average ridership
    average_ridership = day_hour_filtered['ridership'].mean()

    return average_ridership



In [7]:
# Example usage
station_complex_id = '283'  # Example Station ID or part of the name
query_time = '2021-01-01 08:00:00'  # Example query time
average_ridership = get_average_ridership_by_station_and_time(df, station_complex_id, query_time)
print(f"Average ridership for station {station_complex_id} on the same day of the week and time: {average_ridership}")

Average ridership for station 283 on the same day of the week and time: 65.64814814814815


In [8]:
earliest_timestamp = df['transit_timestamp'].min()
earliest_row = df[df['transit_timestamp'] == earliest_timestamp]

# Get the latest transit_timestamp
latest_timestamp = df['transit_timestamp'].max()
latest_row = df[df['transit_timestamp'] == latest_timestamp]

print("Earliest time:")
print(earliest_row["transit_timestamp"].unique()[0])
print("\nLatest time:")
print(latest_row["transit_timestamp"].unique()[0])

print("\nEarliest row(s):")
print(earliest_row)
print("\nLatest row(s):")
print(latest_row)

Earliest time:
2022-02-01 00:00:00

Latest time:
2024-04-11 23:00:00

Earliest row(s):
        transit_timestamp transit_mode station_complex_id  \
4042998        2022-02-01       subway                225   
4047026        2022-02-01       subway                225   
4069597        2022-02-01       subway                225   
4077004        2022-02-01       subway                225   
4088989        2022-02-01       subway                228   
...                   ...          ...                ...   
4871582        2022-02-01       subway                247   
4913825        2022-02-01       subway                248   
4926680        2022-02-01       subway                248   
4956322        2022-02-01       subway                249   
4995653        2022-02-01       subway                250   

                             station_complex    borough payment_method  \
4042998  47-50 Sts-Rockefeller Ctr (B,D,F,M)  Manhattan      metrocard   
4047026  47-50 Sts-Rockefeller C

In [9]:
# Playing around with data
import random

boroughs = df["borough"].unique()
print(boroughs)

# filtered_df = df[(df['borough'] == boroughs[random.randint(0, len(boroughs)-1)])]
borough_filtered = df[(df['borough'] == "Manhattan")]
borough_filtered.head(30)

['Bronx' 'Brooklyn' 'Manhattan' 'Queens' 'Staten Island']


,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
4,2023-09-09 21:00:00,subway,392,"125 St (4,5,6)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,31.0,0.0,40.80414,-73.93759,"{'type': 'Point', 'coordinates': [-73.93759, 4...",2095,749,794
5,2023-09-09 21:00:00,subway,393,116 St (6),Manhattan,metrocard,Metrocard - Seniors & Disability,3.0,0.0,40.79863,-73.94162,"{'type': 'Point', 'coordinates': [-73.94162, 4...",2095,749,756
6,2023-09-09 21:00:00,subway,414,"Bowling Green (4,5)",Manhattan,metrocard,Metrocard - Seniors & Disability,5.0,0.0,40.70482,-74.01407,"{'type': 'Point', 'coordinates': [-74.01407, 4...",2095,749,720
10,2023-09-09 21:00:00,subway,440,"116 St (2,3)",Manhattan,metrocard,Metrocard - Seniors & Disability,5.0,0.0,40.802097,-73.94962,"{'type': 'Point', 'coordinates': [-73.94962, 4...",2095,749,753
17,2023-09-09 21:00:00,subway,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,225.0,0.0,40.749718,-73.98782,"{'type': 'Point', 'coordinates': [-73.98782, 4...",2095,749,717
18,2023-09-09 21:00:00,subway,609,"Bryant Pk (B,D,F,M)/5 Av (7)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,96.0,0.0,40.754223,-73.981964,"{'type': 'Point', 'coordinates': [-73.981964, ...",2095,749,578
20,2023-09-09 21:00:00,subway,610,"Grand Central-42 St (S,4,5,6,7)",Manhattan,metrocard,Metrocard - Full Fare,303.0,0.0,40.751778,-73.976845,"{'type': 'Point', 'coordinates': [-73.976845, ...",2095,749,590
21,2023-09-09 21:00:00,subway,613,"Lexington Av (N,R,W)/59 St (4,5,6)",Manhattan,metrocard,Metrocard - Fair Fare,23.0,3.0,40.76266,-73.967255,"{'type': 'Point', 'coordinates': [-73.967255, ...",2095,749,749
22,2023-09-09 21:00:00,subway,613,"Lexington Av (N,R,W)/59 St (4,5,6)",Manhattan,metrocard,Metrocard - Seniors & Disability,14.0,2.0,40.76266,-73.967255,"{'type': 'Point', 'coordinates': [-73.967255, ...",2095,749,749
23,2023-09-09 21:00:00,subway,619,"Broadway-Lafayette St (B,D,F,M)/Bleecker St (6)",Manhattan,metrocard,Metrocard - Seniors & Disability,9.0,0.0,40.725914,-73.99466,"{'type': 'Point', 'coordinates': [-73.99466, 4...",2095,749,736


In [10]:
station_filted = df[(df['station_complex'] == "7 Av (E,B,D)")]
station_filted

,transit_timestamp,transit_mode,station_complex_id,station_complex,borough,payment_method,fare_class_category,ridership,transfers,latitude,longitude,georeference,:@computed_region_kjdx_g34t,:@computed_region_yamh_8v7k,:@computed_region_wbg7_3whc
1554,2023-09-09 22:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Fair Fare,9.0,1.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
1555,2023-09-09 22:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,58.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
1838,2023-09-09 04:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,3.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
2122,2023-09-09 08:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,14.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
2723,2023-09-09 17:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Seniors & Disability,19.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4039994,2024-04-06 21:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Fair Fare,8.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
4040940,2024-04-06 10:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,65.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
4041641,2024-04-06 00:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 30-Day,35.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742
4041804,2024-04-06 16:00:00,subway,277,"7 Av (E,B,D)",Manhattan,metrocard,Metrocard - Unlimited 7-Day,155.0,0.0,40.762863,-73.981636,"{'type': 'Point', 'coordinates': [-73.981636, ...",2095,749,742


In [11]:
# Preprocessing
aggregations = {
    'transit_timestamp': 'first',
    'ridership': 'sum',  # Sum the ridership for each group
    'transfers': 'sum',  # Also summing the transfers if needed
    # For all other columns that you want to keep the same and which do not have multiple different values per group, you can use 'first' or 'max'
    
    'transit_mode': 'first',
    'borough': 'first',
    'latitude': 'first',
    'longitude': 'first',
    'georeference': 'first'
}

# Perform the groupby and aggregate
condensed_df = df.groupby(['station_complex_id', 'station_complex', 'transit_timestamp'], as_index=False).agg(aggregations)
condensed_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference
0,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 01:00:00,1.0,0.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308..."
1,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 05:00:00,4.0,1.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308..."
2,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 00:00:00,7.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
3,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 01:00:00,8.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
4,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 02:00:00,4.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
...,...,...,...,...,...,...,...,...,...,...
827264,TRAM2,RI Tramway (Roosevelt),2024-04-11 19:00:00,52.0,0.00.00.01.00.01.00.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
827265,TRAM2,RI Tramway (Roosevelt),2024-04-11 20:00:00,85.0,1.00.00.00.08.00.00.00.00.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
827266,TRAM2,RI Tramway (Roosevelt),2024-04-11 21:00:00,37.0,0.00.00.00.08.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."
827267,TRAM2,RI Tramway (Roosevelt),2024-04-11 22:00:00,47.0,0.01.00.00.01.02.0,tram,Manhattan,40.75734,-73.95412,"{'type': 'Point', 'coordinates': [-73.95412, 4..."


In [12]:
# Filtering for subway data
subway_df = condensed_df[condensed_df['transit_mode'] == 'subway']
subway_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference
0,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 01:00:00,1.0,0.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308..."
1,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 05:00:00,4.0,1.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308..."
2,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 00:00:00,7.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
3,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 01:00:00,8.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
4,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 02:00:00,4.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4..."
...,...,...,...,...,...,...,...,...,...,...
824464,99,"Lorimer St (M,J)",2024-04-11 19:00:00,161.0,0.00.00.01.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
824465,99,"Lorimer St (M,J)",2024-04-11 20:00:00,97.0,0.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
824466,99,"Lorimer St (M,J)",2024-04-11 21:00:00,72.0,0.00.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."
824467,99,"Lorimer St (M,J)",2024-04-11 22:00:00,68.0,0.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4..."


In [13]:

final_df = subway_df.dropna(subset=['ridership'])
final_df['month'] = final_df['transit_timestamp'].dt.month
final_df['day_of_week'] = final_df['transit_timestamp'].dt.dayofweek
final_df['hour'] = final_df['transit_timestamp'].dt.hour
final_df

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,month,day_of_week,hour
0,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 01:00:00,1.0,0.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308...",2,1,1
1,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 05:00:00,4.0,1.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308...",2,1,5
2,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 00:00:00,7.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",9,5,0
3,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 01:00:00,8.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",9,5,1
4,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 02:00:00,4.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",9,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
824464,99,"Lorimer St (M,J)",2024-04-11 19:00:00,161.0,0.00.00.01.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",4,3,19
824465,99,"Lorimer St (M,J)",2024-04-11 20:00:00,97.0,0.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",4,3,20
824466,99,"Lorimer St (M,J)",2024-04-11 21:00:00,72.0,0.00.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",4,3,21
824467,99,"Lorimer St (M,J)",2024-04-11 22:00:00,68.0,0.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",4,3,22


In [14]:
# Extracting relavent features and target variable
X = final_df[['hour', 'day_of_week', 'month']]
y = final_df['ridership']

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
# Model Training
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [16]:
predictions = model.predict(X_test)

In [17]:
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Root Mean Square Error: {rmse}")

Root Mean Square Error: 515.1859731102319


In [18]:
# Predictions
def predict_ridership(hour, day_of_week, month):
    input_data = np.array([[hour, day_of_week, month]])
    predicted_ridership = model.predict(input_data)
    return predicted_ridership[0]

hour = 8  # 8 AM
day_of_week = 4  # Friday
month = 1  # January
predicted_ridership = predict_ridership(hour, day_of_week, month)
print(f"Predicted ridership: {predicted_ridership}")

Predicted ridership: 135.27769338208682


/Users/ishaanpoojari/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [19]:
fridays_in_january_at_8am = final_df[(final_df['day_of_week'] == 4) & (final_df['month'] == 1) & (final_df['hour'] == 8)]
fridays_in_january_at_8am

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,month,day_of_week,hour
763,1,"Astoria-Ditmars Blvd (N,W)",2024-01-05 08:00:00,612.0,1.019.00.021.05.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,4,8
2373,10,"49 St (N,R,W)",2024-01-05 08:00:00,122.0,3.00.0,subway,Manhattan,40.7599,-73.98414,"{'type': 'Point', 'coordinates': [-73.98414, 4...",1,4,8
3986,100,"Hewes St (M,J)",2024-01-05 08:00:00,41.0,0.00.00.0,subway,Brooklyn,40.70687,-73.95343,"{'type': 'Point', 'coordinates': [-73.95343, 4...",1,4,8
5588,101,"Marcy Av (M,J,Z)",2024-01-05 08:00:00,171.0,1.07.00.0,subway,Brooklyn,40.70836,-73.957756,"{'type': 'Point', 'coordinates': [-73.957756, ...",1,4,8
7150,103,"Bowery (J,Z)",2024-01-05 08:00:00,17.0,0.00.00.0,subway,Manhattan,40.72028,-73.99391,"{'type': 'Point', 'coordinates': [-73.99391, 4...",1,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
817152,95,"Gates Av (J,Z)",2024-01-05 08:00:00,77.0,3.00.0,subway,Brooklyn,40.68963,-73.92227,"{'type': 'Point', 'coordinates': [-73.92227, 4...",1,4,8
818762,96,Kosciuszko St (J),2024-01-05 08:00:00,17.0,0.0,subway,Brooklyn,40.69334,-73.92882,"{'type': 'Point', 'coordinates': [-73.92882, 4...",1,4,8
820381,97,"Myrtle Av (M,J,Z)",2024-01-05 08:00:00,32.0,1.0,subway,Brooklyn,40.69721,-73.93565,"{'type': 'Point', 'coordinates': [-73.93565, 4...",1,4,8
822007,98,"Flushing Av (M,J)",2024-01-05 08:00:00,49.0,0.00.01.0,subway,Brooklyn,40.70026,-73.941124,"{'type': 'Point', 'coordinates': [-73.941124, ...",1,4,8


In [20]:
final_df.to_json("HistoricalRidership.json", orient='records', lines=True)

print("DataFrame has been successfully saved as JSON.")

DataFrame has been successfully saved as JSON.


In [21]:
final_df2 = subway_df.dropna(subset=['ridership'])
final_df2['station_complex_id'] = final_df2['station_complex_id'].astype(int)

# Feature Engineering
final_df2['hour'] = final_df2['transit_timestamp'].dt.hour
final_df2['day_of_week'] = final_df2['transit_timestamp'].dt.dayofweek
final_df2


,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,hour,day_of_week
0,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 01:00:00,1.0,0.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308...",1,1
1,1,"Astoria-Ditmars Blvd (N,W)",2022-02-01 05:00:00,4.0,1.0,subway,Queens,40.7750358581543,-73.91203308105469,"{'type': 'Point', 'coordinates': [-73.91203308...",5,1
2,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 00:00:00,7.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",0,5
3,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 01:00:00,8.0,0.00.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",1,5
4,1,"Astoria-Ditmars Blvd (N,W)",2023-09-09 02:00:00,4.0,0.0,subway,Queens,40.775036,-73.91203,"{'type': 'Point', 'coordinates': [-73.91203, 4...",2,5
...,...,...,...,...,...,...,...,...,...,...,...,...
824464,99,"Lorimer St (M,J)",2024-04-11 19:00:00,161.0,0.00.00.01.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",19,3
824465,99,"Lorimer St (M,J)",2024-04-11 20:00:00,97.0,0.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",20,3
824466,99,"Lorimer St (M,J)",2024-04-11 21:00:00,72.0,0.00.00.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",21,3
824467,99,"Lorimer St (M,J)",2024-04-11 22:00:00,68.0,0.00.00.00.00.0,subway,Brooklyn,40.70387,-73.94741,"{'type': 'Point', 'coordinates': [-73.94741, 4...",22,3


In [22]:
# Select relevant features for the model, now excluding 'month'
X = final_df2[['station_complex_id', 'day_of_week', 'hour']]
y = final_df2['ridership']

In [23]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predict on the test set
predictions = model.predict(X_test)

In [39]:
# Evaluate the model
mse = mean_squared_error(y_test, predictions)
rmse = np.sqrt(mse)
print(f"Root Mean Square Error: {rmse}")

def predict_ridership(station_complex_id, day_of_week, hour):
    # Format the input features as a DataFrame
    input_features = pd.DataFrame({
        'station_complex_id': [station_complex_id],
        'day_of_week': [day_of_week],
        'hour': [hour]
    })
    # Predict ridership
    predicted_ridership = model.predict(input_features)
    return predicted_ridership[0]

# Example usage
station_id_example = 283  # Replace with an actual station_complex_id
day_of_week_example = 6  # Friday
hour_example = 3  # 8 AM
prediction = predict_ridership(station_id_example, day_of_week_example, hour_example)
print(f"Predicted ridership for station ID {station_id_example} on day {day_of_week_example} at hour {hour_example}: {prediction}")

Root Mean Square Error: 254.00736915219284
Predicted ridership for station ID 283 on day 6 at hour 3: 13.298219036467481


In [40]:
actual_data = final_df2[(final_df2['station_complex_id'] == 283) & (final_df2['day_of_week'] == 6) & (final_df2['hour'] == 3)]
actual_data['ridership'].mean()

11.777777777777779

In [26]:
final_df2[final_df2["station_complex_id"] == 283]

,station_complex_id,station_complex,transit_timestamp,ridership,transfers,transit_mode,borough,latitude,longitude,georeference,hour,day_of_week
400732,283,Greenpoint Av (G),2023-09-09 00:00:00,5.0,0.00.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",0,5
400733,283,Greenpoint Av (G),2023-09-09 03:00:00,1.0,0.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",3,5
400734,283,Greenpoint Av (G),2023-09-09 04:00:00,3.0,0.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",4,5
400735,283,Greenpoint Av (G),2023-09-09 05:00:00,4.0,0.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",5,5
400736,283,Greenpoint Av (G),2023-09-09 07:00:00,13.0,0.00.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",7,5
...,...,...,...,...,...,...,...,...,...,...,...,...
402297,283,Greenpoint Av (G),2024-04-11 19:00:00,422.0,0.00.00.06.00.01.00.01.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",19,3
402298,283,Greenpoint Av (G),2024-04-11 20:00:00,84.0,1.00.00.00.01.00.00.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",20,3
402299,283,Greenpoint Av (G),2024-04-11 21:00:00,212.0,0.00.00.00.00.03.00.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",21,3
402300,283,Greenpoint Av (G),2024-04-11 22:00:00,167.0,0.00.00.01.00.00.00.0,subway,Brooklyn,40.731354,-73.95445,"{'type': 'Point', 'coordinates': [-73.95445, 4...",22,3


In [27]:
errors = abs(predictions - y_test)
mape = 100 * (errors / y_test)
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
mape

Accuracy: -59.92 %.


283122    23.969413
751052     2.051626
340007    33.812062
526558     8.156201
725775     2.956160
            ...    
737914    41.920927
502848    37.573796
769641    63.111178
7010      48.589235
332630    36.025948
Name: ridership, Length: 164284, dtype: float64

In [28]:
# Stations with the most data - NOT the most ridership
station_counts = final_df2['station_complex_id'].value_counts().reset_index()
station_names = final_df2[['station_complex_id', 'station_complex']].drop_duplicates()
station_counts_with_names = station_counts.merge(station_names, on='station_complex_id', how='left')

print(station_counts_with_names.head(25))


    station_complex_id  count  \
0                  243   9614   
1                  232   9602   
2                  225   9581   
3                  234   9567   
4                  228   9552   
5                  237   9539   
6                  236   9538   
7                  235   9512   
8                  231   9501   
9                  238   9497   
10                 240   9486   
11                 241   9479   
12                 242   9394   
13                 245   8591   
14                 244   8511   
15                 249   8219   
16                 248   8171   
17                 246   8126   
18                 247   7829   
19                 250   7722   
20                 158   3601   
21                  62   2239   
22                 255   1906   
23                 611   1730   
24                 610   1694   

                                     station_complex  
0                                    Church Av (F,G)  
1                              

In [29]:
top_10_station_ids = station_counts_with_names['station_complex_id'].head(10).tolist()
top_10_station_ids

[243, 232, 225, 234, 228, 237, 236, 235, 231, 238]

In [30]:
# Repeated trials testing

def test_ridership_predictions(station_ids, days_of_week, hours):
    results = []

    for station_id in station_ids:
        for day in days_of_week:
            for hour in hours:
                # Predict ridership
                prediction = predict_ridership(station_id, day, hour)

                # Extract actual data
                actual_data = final_df2[
                    (final_df2['station_complex_id'] == station_id) &
                    (final_df2['day_of_week'] == day) &
                    (final_df2['hour'] == hour)
                ]

                # Compute actual mean and median ridership
                actual_mean = actual_data['ridership'].mean()
                actual_median = actual_data['ridership'].median()

                # Calculate the percentage error between predicted and actual mean
                percentage_error_mean = 100 * abs(prediction - actual_mean) / actual_mean if actual_mean > 0 else None
                # Calculate the percentage error between predicted and actual median
                percentage_error_median = 100 * abs(prediction - actual_median) / actual_median if actual_median > 0 else None

                # Record the results
                results.append({
                    'Station ID': station_id,
                    'Day of Week': day,
                    'Hour': hour,
                    'Predicted Ridership': prediction,
                    'Actual Mean Ridership': actual_mean,
                    'Actual Median Ridership': actual_median,
                    'Percentage Error (Mean)': percentage_error_mean,
                    'Percentage Error (Median)': percentage_error_median
                })

                # Print the results for each prediction
                print(f"Station ID {station_id}, Day {day}, Hour {hour}")
                print(f"Predicted Ridership: {prediction}")
                print(f"Actual Mean Ridership: {actual_mean}")
                print(f"Actual Median Ridership: {actual_median}")
                print(f"Percentage Error (Mean): {percentage_error_mean}%")
                print(f"Percentage Error (Median): {percentage_error_median}%")
                print('-------------------------------------')

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Compute overall percentage error
    overall_percentage_error_mean = results_df['Percentage Error (Mean)'].mean()
    overall_percentage_error_median = results_df['Percentage Error (Median)'].mean()

    print(f"Overall Percentage Error (Mean): {overall_percentage_error_mean}%")
    print(f"Overall Percentage Error (Median): {overall_percentage_error_median}%")

    return results_df

# Example usage with a list of station IDs, days, and hours
station_ids = top_10_station_ids  # Replace with your actual list of station IDs
days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0 = Monday, 6 = Sunday
hours = [8, 14, 20]  # Example set of hours in 24-hour format

# Call the test function
test_results_df = test_ridership_predictions(station_ids, days_of_week, hours)


Station ID 243, Day 0, Hour 8
Predicted Ridership: 793.6834705443302
Actual Mean Ridership: 769.6428571428571
Actual Median Ridership: 886.0
Percentage Error (Mean): 3.1236063816299207%
Percentage Error (Median): 10.41947285052706%
-------------------------------------
Station ID 243, Day 0, Hour 14
Predicted Ridership: 274.3928828016957
Actual Mean Ridership: 284.89285714285717
Actual Median Ridership: 293.5
Percentage Error (Mean): 3.6855870822680368%
Percentage Error (Median): 6.510091038604535%
-------------------------------------
Station ID 243, Day 0, Hour 20
Predicted Ridership: 112.64816537922007
Actual Mean Ridership: 112.49090909090908
Actual Median Ridership: 112.0
Percentage Error (Mean): 0.1397946639260454%
Percentage Error (Median): 0.5787190885893523%
-------------------------------------
Station ID 243, Day 1, Hour 8
Predicted Ridership: 912.6953083064931
Actual Mean Ridership: 932.4736842105264
Actual Median Ridership: 1016.0
Percentage Error (Mean): 2.121065316795351

In [31]:

def test_ridership_predictions(station_ids, days_of_week, hours):
    results = []

    for station_id in station_ids:
        for day in days_of_week:
            for hour in hours:
                # Predict ridership
                prediction = predict_ridership(station_id, day, hour)

                # Extract actual data
                actual_data = final_df2[
                    (final_df2['station_complex_id'] == station_id) &
                    (final_df2['day_of_week'] == day) &
                    (final_df2['hour'] == hour)
                ]

                # Compute actual mean and median ridership
                actual_mean = actual_data['ridership'].mean()
                actual_median = actual_data['ridership'].median()

                # Calculate the percentage error between predicted and actual mean
                percentage_error_mean = 100 * abs(prediction - actual_mean) / actual_mean if actual_mean > 0 else None
                # Calculate the percentage error between predicted and actual median
                percentage_error_median = 100 * abs(prediction - actual_median) / actual_median if actual_median > 0 else None

                # Record the results
                results.append({
                    'Station ID': station_id,
                    'Day of Week': day,
                    'Hour': hour,
                    'Predicted Ridership': prediction,
                    'Actual Mean Ridership': actual_mean,
                    'Actual Median Ridership': actual_median,
                    'Percentage Error (Mean)': percentage_error_mean,
                    'Percentage Error (Median)': percentage_error_median
                })

                # Print the results for each prediction
                print(f"Station ID {station_id}, Day {day}, Hour {hour}")
                print(f"Predicted Ridership: {prediction}")
                print(f"Actual Mean Ridership: {actual_mean}")
                print(f"Actual Median Ridership: {actual_median}")
                print(f"Percentage Error (Mean): {percentage_error_mean}%")
                print(f"Percentage Error (Median): {percentage_error_median}%")
                print('-------------------------------------')

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Compute overall percentage error
    overall_percentage_error_mean = results_df['Percentage Error (Mean)'].mean()
    overall_percentage_error_median = results_df['Percentage Error (Median)'].mean()

    print(f"Overall Percentage Error (Mean): {overall_percentage_error_mean}%")
    print(f"Overall Percentage Error (Median): {overall_percentage_error_median}%")

    return results_df

# Example usage with a list of station IDs, days, and hours
station_ids = top_10_station_ids  # Replace with your actual list of station IDs
days_of_week = [0, 1, 2, 3, 4, 5, 6]  # 0 = Monday, 6 = Sunday
hours = [8, 14, 20]  # Example set of hours in 24-hour format

# Call the test function
test_results_df = test_ridership_predictions(station_ids, days_of_week, hours)

Station ID 243, Day 0, Hour 8
Predicted Ridership: 793.6834705443302
Actual Mean Ridership: 769.6428571428571
Actual Median Ridership: 886.0
Percentage Error (Mean): 3.1236063816299207%
Percentage Error (Median): 10.41947285052706%
-------------------------------------
Station ID 243, Day 0, Hour 14
Predicted Ridership: 274.3928828016957
Actual Mean Ridership: 284.89285714285717
Actual Median Ridership: 293.5
Percentage Error (Mean): 3.6855870822680368%
Percentage Error (Median): 6.510091038604535%
-------------------------------------
Station ID 243, Day 0, Hour 20
Predicted Ridership: 112.64816537922007
Actual Mean Ridership: 112.49090909090908
Actual Median Ridership: 112.0
Percentage Error (Mean): 0.1397946639260454%
Percentage Error (Median): 0.5787190885893523%
-------------------------------------
Station ID 243, Day 1, Hour 8
Predicted Ridership: 912.6953083064931
Actual Mean Ridership: 932.4736842105264
Actual Median Ridership: 1016.0
Percentage Error (Mean): 2.121065316795351

In [33]:
average_ridership_per_station = final_df2.groupby(['station_complex_id', 'station_complex'])['ridership'].mean().reset_index()
sorted_stations_by_ridership = average_ridership_per_station.sort_values(by='ridership', ascending=False)
sorted_stations_by_ridership
top_10_station_ids = sorted_stations_by_ridership['station_complex_id'].head(10).tolist()

# Print the list of the top 10 station IDs
print(top_10_station_ids)
sorted_stations_by_ridership.head(25)

[611, 610, 602, 607, 628, 164, 614, 616, 318, 447]


,station_complex_id,station_complex,ridership
402,611,"Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",3784.447977
401,610,"Grand Central-42 St (S,4,5,6,7)",2654.181818
393,602,"14 St-Union Sq (L,N,Q,R,W,4,5,6)",1921.021486
398,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",1880.625759
419,628,"Fulton St (A,C,J,Z,2,3,4,5)",1564.301703
130,164,"34 St-Penn Station (A,C,E)",1550.535135
405,614,"59 St-Columbus Circle (A,B,C,D,1)",1434.055283
407,616,"74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,...",1371.438659
262,318,"34 St-Penn Station (1,2,3)",1333.591592
372,447,Flushing-Main St (7),1252.677477


In [38]:
from joblib import dump

# Save model
dump(model, 'models/fiveMillionDatapointModel.joblib')

['fiveMillionDatapointModel.joblib']

In [41]:
#Might be too large
# final_df2.to_json("HistoricalRidership.json", orient='records', lines=True)

# print("DataFrame has been successfully saved as JSON.")

DataFrame has been successfully saved as JSON.


In [1]:
sorted_stations_by_ridership

NameError: name 'sorted_stations_by_ridership' is not defined